<a href="https://colab.research.google.com/github/ammarhusain/XCS229ii-project/blob/main/rl_env_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Custom gym environment

- Sets up a custom & barebones OpenAI gym environment 
- Uses an off the shelf RL algorithm from stable baselines 3 to plugin and solve a simple grid search problem.


In [ ]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!pip install stable-baselines[mpi]==2.10.0

TensorFlow 1.x selected.
     |████████████████████████████████| 248 kB 16.4 MB/s 
  Attempting uninstall: stable-baselines
    Found existing installation: stable-baselines 2.2.1
    Uninstalling stable-baselines-2.2.1:
      Successfully uninstalled stable-baselines-2.2.1


In [ ]:
import numpy as np
import gym
from gym import spaces


class HypRLGridEnv(gym.Env):
  """
  Custom Environment that follows gym interface.
  This is a simple env where the agent must learn to go always left. 
  """
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {'render.modes': ['console']}
  # Define constants for clearer code
  UP = 0
  DOWN = 1
  MAX_ITER = 10
  # second dimenstion on how the agent is progressing
  LOWER=0
  SAME=1
  HIGHER=2
  ## hardcoded ground truth reward
  GROUND_TRUTH = [-5,-4,-3,-2,-1,0,-1,-2,-3,-4,-5]
  def __init__(self, grid_size=11):
    super(HypRLGridEnv, self).__init__()

    # Size of the 1D-grid
    self.grid_size = grid_size
    self.grid_shape = [grid_size, 3]
    # Initialize the agent randomly
    self.agent_pos = np.random.randint(grid_size)

    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions, we have two: left and right
    n_actions = 2
    self.action_space = spaces.Discrete(n_actions)
    # The observation will be the coordinate of the agent
    # this can be described both by Discrete and Box space
    self.observation_space = spaces.MultiDiscrete([grid_size, 3])

    #self.reset()

  def reset(self):
    """
    Important: the observation must be a numpy array
    :return: (np.array) 
    """
    # reset the number of iterations for this agent
    self.iter = 0
    self.reward = -np.inf
    # Initialize the agent at the right of the grid
    self.agent_pos = np.random.randint(self.grid_size)

    start_state = [self.agent_pos, 1]
    self.visited = [start_state]
    return np.array(start_state) # same bucket

  def step(self, action):
    self.iter += 1
    prev_grid_val =  self.GROUND_TRUTH[self.agent_pos]
    curr_pos = self.agent_pos
    if action == self.UP:
      self.agent_pos -= 1
    elif action == self.DOWN:
      self.agent_pos += 1
    else:
      raise ValueError("Received invalid action={} which is not part of the action space".format(action))
    # Account for the boundaries of the grid
    self.agent_pos = np.clip(self.agent_pos, 0, self.grid_size-1)

    curr_grid_val =  self.GROUND_TRUTH[self.agent_pos]
    # Figure out where the agent falls in the high,same,low bucket
    high_low_bucket = self.SAME
    if prev_grid_val < curr_grid_val:
      high_low_bucket = self.HIGHER
    elif prev_grid_val > curr_grid_val:
      high_low_bucket = self.LOWER

    # set the new state
    new_state = [self.agent_pos, high_low_bucket]

    # We are done when we visit the same state twice or have taken more iterations than MAX
    done = bool(self.iter >= self.MAX_ITER or new_state in self.visited)

    self.visited.append(new_state)

    # Reward is minimum of whatever val loss we saw so far
    self.reward = max(self.reward, curr_grid_val)

    # Null reward everywhere except when the episode terminates
    reward = self.reward if done else 0

    # Optionally we can pass additional info, we are not using that for now
    info = {}
    return np.array(new_state), reward, done, info

  def render(self, mode='console'):
    if mode != 'console':
      raise NotImplementedError()
    # agent is represented as a cross, rest as a dot
    print("." * self.agent_pos, end="")
    print("x", end="")
    print("." * (self.grid_size - self.agent_pos))

  def close(self):
    pass

# check and make sure the environment is sane and working
from stable_baselines.common.env_checker import check_env

env = HypRLGridEnv()
# If the environment doesn't follow the interface, an error will be thrown
check_env(env, warn=True)
env.render()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

.....x......


In [ ]:
from stable_baselines import DQN, PPO2, A2C, ACKTR
from stable_baselines.common.cmd_util import make_vec_env
from stable_baselines.common.policies import MlpPolicy
import pdb
# Instantiate the env
env = HypRLGridEnv()
# wrap it
env = make_vec_env(lambda: env, n_envs=1)

# Train the agent
#model = ACKTR('MlpPolicy', env, verbose=1).learn(5000)
model = A2C(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=25000)
# Test the trained agent
obs = env.reset()
n_steps = 20
for step in range(n_steps):
  action, _ = model.predict(obs, deterministic=True)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  #pdb.set_trace()
  obs, reward, done, info = env.step(action)
  print(obs, ' info=', info)
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    print("Goal reached!", "reward=", reward, "final_state=", info[0]['terminal_observation'])
    break
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render(mode='console')





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


---------------------------------
| ep_len_mean        | 3        |
| ep_reward_mean     | -2       |
| explained_variance | 0.158    |
| fps                | 12       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 2.01     |
---------------------------------
---------------------------------
| ep_len_mean        | 4.49     |
| ep_reward_mean     | -1.93    |
| explained_variance | -0.25    |
| fps                | 486      |
| nupdates           | 100      |
| policy_entropy     | 0.693    |
| total_timesteps    | 500      |
| value_loss         | 1.96    

## Ammar's experiments



In [ ]:
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 8

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
full_train = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
full_test = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# separate out some training data to train the RL agent
half_data_size = int(len(trainset)/2)

rl_agent_trainset = torch.utils.data.Subset(trainset, range(0,int(0.8*half_data_size)))
rl_agent_testset = torch.utils.data.Subset(trainset, range(int(0.8*half_data_size), half_data_size))


rl_agent_train = torch.utils.data.DataLoader(rl_agent_trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
rl_agent_test = torch.utils.data.DataLoader(rl_agent_testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

print(f"Full dataset size:  train={len(full_train)*batch_size} test={len(full_test)*batch_size}")
print(f"Use a subset of the training data to train the Hyp-RL agent : train={len(rl_agent_train)*batch_size} val={len(rl_agent_test)*batch_size}")


Files already downloaded and verified
Files already downloaded and verified
Full dataset size:  train=50000 test=10000
Use a subset of the training data to train the Hyp-RL agent : train=20000 val=5000


## Ammar's XCS229ii experiments




In [ ]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!pip install stable-baselines[mpi]==2.10.0

import matplotlib.pyplot as plt
import numpy as np

# function to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


TensorFlow 1.x selected.
     |████████████████████████████████| 248 kB 5.0 MB/s 
  Attempting uninstall: stable-baselines
    Found existing installation: stable-baselines 2.2.1
    Uninstalling stable-baselines-2.2.1:
      Successfully uninstalled stable-baselines-2.2.1


NameError: ignored

In [ ]:
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 8

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# separate out some training data to train the RL agent
half_data_size = int(len(trainset)/2)

rl_agent_trainset = torch.utils.data.Subset(trainset, range(0,int(0.8*half_data_size)))
rl_agent_testset = torch.utils.data.Subset(trainset, range(int(0.8*half_data_size), half_data_size))

print(f"Full dataset size:  train={len(trainset)} test={len(testset)}")
print(f"Use a subset of the training data to train the Hyp-RL agent : train={len(rl_agent_trainset)} val={len(rl_agent_testset)}")


Files already downloaded and verified
Files already downloaded and verified
Full dataset size:  train=50000 test=10000
Use a subset of the training data to train the Hyp-RL agent : train=20000 val=5000


In [ ]:
## function to train and evaluate the model given the hyperparameter setting

## define the neural network
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def evaluateFullDataset(hp_learning_rate=0.001):
  full_train = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

  full_test = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
  net = Net()
  loss_criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=hp_learning_rate, momentum=0.9)
  trainAndEvaluateModel(net, loss_criterion, optimizerm, rl_agent_train, rl_agent_test)

def trainAndEvaluateModel(net, loss_criterion, optimizer, train, test):
  ## Train the model
  for epoch in range(10):  # loop over the dataset multiple times

      running_loss = 0.0
      for i, data in enumerate(train, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)
          loss = loss_criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          if i % 2000 == 1999:    # print every 2000 mini-batches
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
              running_loss = 0.0
  print('Finished Training')

  ## Test the model
  dataiter = iter(test)
  images, labels = dataiter.next()

  # print images
  imshow(torchvision.utils.make_grid(images))
  print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

  outputs = net(images)

  correct = 0
  total = 0
  # since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
      for data in test:
          images, labels = data
          # calculate outputs by running images through the network 
          outputs = net(images)
          # the class with the highest energy is what we choose as prediction
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f"Accuracy of the network on the {len(test)} test images: {(100 * correct / total)}%")
  return (100 * correct / total)
  
evaluateRLAgent()
#evaluateFullDataset()

0.001

### Build the RL environment and agent

In [ ]:
import numpy as np
import gym
from gym import spaces

def evaluateRLAgent(hp_learning_rate=0.001):
  #return hp_learning_rate
  rl_agent_train = torch.utils.data.DataLoader(rl_agent_trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
  rl_agent_test = torch.utils.data.DataLoader(rl_agent_testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
  net = Net()
  loss_criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=hp_learning_rate, momentum=0.9)
  trainAndEvaluateModel(net, loss_criterion, optimizer, rl_agent_train, rl_agent_test)
   
class TunableHP:
  def __init__(self):
    self.hyperparameters = {"learning_rate":[0.0001, 0.001, 0.01, 0.1, 1.0]}

    #self.hyperparameters = {"learning_rate":[-5,-4,-3,-2,-1,0,-1,-2,-3,-4,-5]}
    self.hyperparameter_keys = list(self.hyperparameters)

  def mapStateToHP(self,state):
    return [self.hyperparameters[self.hyperparameter_keys[p]][i] for p,i in enumerate(state)]
  
  def getGridSize(self):
    return [len(self.hyperparameters[k]) for k in self.hyperparameter_keys]



class HypRLGridEnv(gym.Env):
  """
  Custom Environment that follows gym interface.
  This is a simple env where the agent must learn to go always left. 
  """
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {'render.modes': ['console']}
  # Define constants for clearer code
  UP = 0
  DOWN = 1
  MAX_ITER = 10

  def __init__(self, tunableParams=TunableHP()):
    super(HypRLGridEnv, self).__init__()

    self.tunableParams = tunableParams

    # Size of the grid
    self.grid_size = tunableParams.getGridSize()
    
    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions, we have two: left and right
    n_actions = 2
    self.action_space = spaces.Discrete(n_actions)
    # The observation will be the coordinate of the agent
    # this can be described both by Discrete and Box space
    self.observation_space = spaces.MultiDiscrete(self.grid_size)

    self.eval_cache = np.zeros(self.grid_size)

  def eval(self, state):
    state = tuple(state)
    if self.eval_cache[state] == [0.0]:
      # train & test the model for these hyperparameters
      self.eval_cache[state] = evaluateRLAgent(*self.tunableParams.mapStateToHP(state))
    return self.eval_cache[state]

  def reset(self):
    """
    Important: the observation must be a numpy array
    :return: (np.array) 
    """
    # reset the number of iterations for this agent
    self.iter = 0
    self.reward = -np.inf
    # Initialize the agent at the right of the grid
    self.agent_state = np.random.randint(self.grid_size)

    start_state = self.agent_state
    self.visited = {}
    return np.array(start_state) 

  def step(self, action):
    self.iter += 1

    if action == self.UP:
      self.agent_state -= 1
    elif action == self.DOWN:
      self.agent_state += 1
    else:
      raise ValueError("Received invalid action={} which is not part of the action space".format(action))
    # Account for the boundaries of the grid
    for i, _ in enumerate(self.agent_state):
      self.agent_state[i] = np.clip(self.agent_state[i], 0, self.grid_size[i]-1)

    # We are done when we visit the same state twice or have taken more iterations than MAX
    done = bool(self.iter >= self.MAX_ITER or tuple(self.agent_state) in self.visited)

    self.visited[tuple(self.agent_state)] = True

    # Reward is minimum of whatever val loss we saw so far
    self.reward = max(self.reward, self.eval(self.agent_state))

    # Null reward everywhere except when the episode terminates
    reward = self.reward if done else 0

    # Optionally we can pass additional info, we are not using that for now
    info = {}
    return np.array(self.agent_state), reward, done, info

  def render(self, mode='console'):
    if mode != 'console':
      raise NotImplementedError()
    # agent is represented as a cross, rest as a dot
    print("." * self.agent_state, end="")
    print("x", end="")
    print("." * (self.grid_size - self.agent_state))

  def close(self):
    pass

# check and make sure the environment is sane and working
from stable_baselines.common.env_checker import check_env

#env = HypRLGridEnv()
# If the environment doesn't follow the interface, an error will be thrown
#check_env(env, warn=True)
#env.render()

### RL Agent

In [ ]:
from stable_baselines import DQN, PPO2, A2C, ACKTR
from stable_baselines.common.cmd_util import make_vec_env
from stable_baselines.common.policies import MlpPolicy
import pdb
# Instantiate the env
env = HypRLGridEnv()
# wrap it
env = make_vec_env(lambda: env, n_envs=1)

# Train the agent
#model = ACKTR('MlpPolicy', env, verbose=1).learn(5000)
model = A2C(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=25000)
# Test the trained agent
obs = env.reset()
n_steps = 20
for step in range(n_steps):
  action, _ = model.predict(obs, deterministic=True)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  #pdb.set_trace()
  obs, reward, done, info = env.step(action)
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    print("Goal reached!", "reward=", reward, "final_state=", info[0]['terminal_observation'])
    break
  print('obs=', obs, 'reward=', reward, 'done=', done)
  #env.render(mode='console')

[1,  2000] loss: 2.624


In [ ]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!pip install stable-baselines[mpi]==2.10.0

import matplotlib.pyplot as plt
import numpy as np

# function to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


TensorFlow 1.x selected.


In [ ]:
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 8

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# separate out some training data to train the RL agent
half_data_size = int(len(trainset)/2)

rl_agent_trainset = torch.utils.data.Subset(trainset, range(0,int(0.8*half_data_size)))
rl_agent_testset = torch.utils.data.Subset(trainset, range(int(0.8*half_data_size), half_data_size))

print(f"Full dataset size:  train={len(trainset)} test={len(testset)}")
print(f"Use a subset of the training data to train the Hyp-RL agent : train={len(rl_agent_trainset)} val={len(rl_agent_testset)}")


Files already downloaded and verified
Files already downloaded and verified
Full dataset size:  train=50000 test=10000
Use a subset of the training data to train the Hyp-RL agent : train=20000 val=5000


In [ ]:
## function to train and evaluate the model given the hyperparameter setting

## define the neural network
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def evaluateFullDataset(hp_learning_rate=0.001):
  full_train = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

  full_test = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
  net = Net()
  loss_criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=hp_learning_rate, momentum=0.9)
  trainAndEvaluateModel(net, loss_criterion, optimizerm, rl_agent_train, rl_agent_test)

def trainAndEvaluateModel(net, loss_criterion, optimizer, train, test):
  ## Train the model
  for epoch in range(2):  # loop over the dataset multiple times

      running_loss = 0.0
      for i, data in enumerate(train, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)
          loss = loss_criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          if i % 2000 == 1999:    # print every 2000 mini-batches
              # print('[%d, %5d] loss: %.3f' %
              #       (epoch + 1, i + 1, running_loss / 2000))
              running_loss = 0.0
  #print('Finished Training')

  ## Test the model

  # # print images
  # dataiter = iter(test)
  # images, labels = dataiter.next()
  # imshow(torchvision.utils.make_grid(images))
  # print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
  # outputs = net(images)

  correct = 0
  total = 0
  # since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
      for data in test:
          images, labels = data
          # calculate outputs by running images through the network 
          outputs = net(images)
          # the class with the highest energy is what we choose as prediction
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f"Accuracy of the network on the {len(test)} test images: {(100 * correct / total)}%")
  return (100 * correct / total)
  

#evaluateFullDataset()

## Build the RL environment and agent

In [ ]:
import numpy as np
import gym
from gym import spaces

def evaluateRLAgent(hp_learning_rate):
  print(f"Running evaluation for : {hp_learning_rate}")
  #return hp_learning_rate
  rl_agent_train = torch.utils.data.DataLoader(rl_agent_trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
  rl_agent_test = torch.utils.data.DataLoader(rl_agent_testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
  net = Net()
  loss_criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=hp_learning_rate, momentum=0.9)
  return trainAndEvaluateModel(net, loss_criterion, optimizer, rl_agent_train, rl_agent_test)
   
class TunableHP:
  def __init__(self):
    self.hyperparameters = {"learning_rate":[0.0001, 0.001, 0.01, 0.1, 1.0]}

    #self.hyperparameters = {"learning_rate":[-5,-4,-3,-2,-1,0,-1,-2,-3,-4,-5]}
    self.hyperparameter_keys = list(self.hyperparameters)

  def mapStateToHP(self,state):
    return [self.hyperparameters[self.hyperparameter_keys[p]][i] for p,i in enumerate(state)]
  
  def getGridSize(self):
    return [len(self.hyperparameters[k]) for k in self.hyperparameter_keys]



class HypRLGridEnv(gym.Env):
  """
  Custom Environment that follows gym interface.
  This is a simple env where the agent must learn to go always left. 
  """
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {'render.modes': ['console']}
  # Define constants for clearer code
  UP = 0
  DOWN = 1
  STAY = 2
  MAX_ITER = 10

  def __init__(self, tunableParams=TunableHP()):
    super(HypRLGridEnv, self).__init__()

    self.tunableParams = tunableParams

    # Size of the grid
    self.grid_size = tunableParams.getGridSize()
    
    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions, we have two: left and right
    n_actions = 3
    self.action_space = spaces.Discrete(n_actions)
    # The observation will be the coordinate of the agent
    # this can be described both by Discrete and Box space
    self.observation_space = spaces.MultiDiscrete(self.grid_size)

    self.eval_cache = np.zeros(self.grid_size)

  def eval(self, state):
    state = tuple(state)
    if self.eval_cache[state] == [0.0]:
      # train & test the model for these hyperparameters
      self.eval_cache[state] = evaluateRLAgent(*self.tunableParams.mapStateToHP(state))
    return self.eval_cache[state]

  def reset(self):
    """
    Important: the observation must be a numpy array
    :return: (np.array) 
    """
    # reset the number of iterations for this agent
    self.iter = 0
    self.reward = -np.inf
    # Initialize the agent at the right of the grid
    self.agent_state = np.random.randint(self.grid_size)

    start_state = self.agent_state
    self.visited = {}
    return np.array(start_state) 

  def step(self, action):
    self.iter += 1

    if action == self.UP:
      self.agent_state -= 1
    elif action == self.DOWN:
      self.agent_state += 1
    elif action == self.STAY:
      self.agent_state = self.agent_state      
    else:
      raise ValueError("Received invalid action={} which is not part of the action space".format(action))
    # Account for the boundaries of the grid
    for i, _ in enumerate(self.agent_state):
      self.agent_state[i] = np.clip(self.agent_state[i], 0, self.grid_size[i]-1)

    # We are done when we visit the same state twice or have taken more iterations than MAX
    done = bool(self.iter >= self.MAX_ITER or tuple(self.agent_state) in self.visited)

    self.visited[tuple(self.agent_state)] = True

    # Reward is minimum of whatever val loss we saw so far
    self.reward = max(self.reward, self.eval(self.agent_state))

    # Null reward everywhere except when the episode terminates
    reward = self.reward if done else 0

    # Optionally we can pass additional info, we are not using that for now
    info = {}
    return np.array(self.agent_state), reward, done, info

  def render(self, mode='console'):
    if mode != 'console':
      raise NotImplementedError()
    # agent is represented as a cross, rest as a dot
    print("." * self.agent_state, end="")
    print("x", end="")
    print("." * (self.grid_size - self.agent_state))

  def close(self):
    pass

# check and make sure the environment is sane and working
from stable_baselines.common.env_checker import check_env

#env = HypRLGridEnv()
# If the environment doesn't follow the interface, an error will be thrown
#check_env(env, warn=True)
#env.render()

### RL Agent

In [ ]:
from stable_baselines import DQN, PPO2, A2C, ACKTR
from stable_baselines.common.cmd_util import make_vec_env
from stable_baselines.common.policies import MlpPolicy
import pdb
# Instantiate the env
env = HypRLGridEnv()
# wrap it
env = make_vec_env(lambda: env, n_envs=1)

# Train the agent
model = ACKTR('MlpPolicy', env, verbose=1).learn(5000)
model = A2C(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=25000)













Running evaluation for : 0.0001
Finished Training
Accuracy of the network on the 625 test images: 18.1%
Running evaluation for : 1.0
Finished Training
Accuracy of the network on the 625 test images: 9.62%
Running evaluation for : 0.001
Finished Training
Accuracy of the network on the 625 test images: 43.22%
Running evaluation for : 0.1
Finished Training
Accuracy of the network on the 625 test images: 9.46%
Running evaluation for : 0.01
Finished Training
Accuracy of the network on the 625 test images: 43.18%
---------------------------------
| ep_len_mean        | 2.43     |
| ep_reward_mean     | 25.2     |
| explained_variance | -0.00226 |
| fps                | 0        |
| nupdates           | 1        |
| policy_entropy     | 1.1      |
| policy_loss        | 26.2     |
| total_timesteps    | 20       |
| value_loss         | 878      |
---------------------------------
---------------------------------
| ep_len_mean        | 3.56     |
| ep_reward_mean     | 42.5     |


In [ ]:
# Test the trained agent
obs = env.reset()
n_steps = 20
for step in range(n_steps):
  action, _ = model.predict(obs, deterministic=True)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  #pdb.set_trace()
  obs, reward, done, info = env.step(action)
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    print("Goal reached!", "reward=", reward, "final_state=", info[0]['terminal_observation'])
    break
  print('obs=', obs, 'reward=', reward, 'done=', done)
  #env.render(mode='console')

Step 1
Action:  [2]
obs= [[1]] reward= [0.] done= [False]
Step 2
Action:  [2]
Goal reached! reward= [43.22] final_state= [1]
